In [2]:
# general purpose modules for handling data
import numpy as np
from numpy import array
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

# for loading telo data column containing individual
# telomere length values
from ast import literal_eval

# custom module for handling telomere length data
import telomere_methods_astros as telo_ma

In [84]:
import importlib
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
&nbsp; 

The dataframe containing our telomere length data already contains the means. However, we're interested in analyzing (graphing, stats) the individual telomere length measurements. To do so we must extract the values from the list, while keeping the values associated with their timepont/individual. We'll achieve this by 'exploding' the cell containing individual telomere length measurements as a list, into one row per measurement (5520 rows per sample, 184 telos per 30 metaphases per sample). 

As well, we're interested in defining and examining the prevalence of short/long telomeres; this is achieved by finding the values that divide the data into bottom 25% (short telos), middle 50%, and top 25% (long telos). More on that soon.
 
&nbsp;
___

### Reading astronaut dataframe from file

In [90]:
astro_df = pd.read_csv('../excel data/All_astronauts_telomere_length_dataframe.csv')

# literal eval enables interpretation of individual telomere length values
# in the list 
astro_df['telo data'] = astro_df['telo data'].apply(lambda row: np.array(literal_eval(row)))

In [93]:
astro_df.head(4)

,astro number,astro id,timepoint,flight status,telo data,telo means,Q1,Q2-3,Q4
0,1,5163,L-270,Pre-Flight,"[46.9762779819579, 78.38289341797528, 105.1119...",87.672120,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75
1,1,5163,L-180,Pre-Flight,"[102.38890745071835, 107.3170731707317, 61.944...",101.077756,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75
2,1,5163,L-60,Pre-Flight,"[92.4323421316405, 144.537253591714, 135.59973...",128.599235,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75
3,1,5163,FD90,Mid-Flight,"[89.22485800200468, 116.6388239224858, 116.221...",101.183129,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75


### Exploding list containing individual telomere length measurements into rows

this action explodes the list into columns bearing each datapoint.
picture the list, which contains the individual values, expanding to the
right, up to 5520 measurements

importantly, the index #s per row still refer to the timepont/sample
so we can merge these columns back to the original astro_df
then we'll melt the columns into one, resulting in a lot of rows
where each has one individual telomere length measurement

In [8]:
explode_telos_raw = astro_df['telo data'].apply(pd.Series)

In [10]:
explode_telos_raw.head(2)

,0,1,2,3,4,5,6,7,8,9,...,5510,5511,5512,5513,5514,5515,5516,5517,5518,5519
0,46.976278,78.382893,105.111928,37.236886,69.111260,101.620448,59.321751,61.443368,85.633144,80.838623,...,58.453057,114.049449,135.967257,68.242566,101.553625,140.678249,82.308720,69.027731,54.376879,69.395256
1,102.388907,107.317073,61.944537,108.102239,94.854661,91.764116,77.464083,114.316739,131.440027,89.876378,...,106.698964,66.471767,41.430003,89.909790,110.892082,83.578349,101.470097,163.431340,116.839292,44.136318


In [13]:
exploded_telos_astro_df = (explode_telos_raw
    # merge 5520 columns with original dataframe
    .merge(astro_df, right_index = True, left_index = True)
                           
    # drop unnecessary columns
    .drop(['telo data', 'Q1', 'Q2-3', 'Q4'], axis = 1)
                           
    #specify which columns remain constant per indviidual telo 
    .melt(id_vars = ['astro number', 'astro id', 'timepoint', 'flight status', 'telo means'], value_name = "telo data exploded") 
    .drop("variable", axis = 1)
    .dropna())

exploded_telos_astro_df.shape

(436080, 6)

### Saving exploded individual telos for all astros dataframe to csv for later retrieval

In [15]:
copy_exploded_telos_astros_df = exploded_telos_astro_df

copy_exploded_telos_astros_df.to_csv('../excel data/exploded_telos_astros_df.csv', index = False)

### Defining short/mid/long telomeres w/ quartiles and counting per timepoint per astro

This function identifies the timepoint per astronaut most distal to spaceflight (L-270 or L-180) and identifies the individual telomere length values which divide the data into the bottom 25% (short telos), mid 50%, and top 25% (long telos). The function then counts how many telos reside in each quartile. Now we have a way to define short/long telomeres.

The function then applies those quartile cutoff values to subsequent datapoints and counts how many telomeres reside within each quartile. In doing so, we count the number of telomeres moving into/out of the quartiles, per timepoint per astronaut, and can quantitatively dicuss # of short/long telos.

In [142]:
quartiles_astro_df = telo_ma.make_quartiles_columns(astro_df)

In [141]:
%reload_ext autoreload

In [144]:
# for col in ['Q1', 'Q2-3', 'Q4']:
#     quartiles_astro_df[col] = quartiles_astro_df[col].astype('int64')

quartiles_astro_df['Q1'] = quartiles_astro_df['Q1'].astype('int64')
quartiles_astro_df['Q2-3'] = quartiles_astro_df['Q2-3'].astype('int64')
quartiles_astro_df['Q4'] = quartiles_astro_df['Q4'].astype('int64')

quartiles_astro_df[quartiles_astro_df['astro id'] == 2171]

,astro number,astro id,timepoint,flight status,telo data,telo means,Q1,Q2-3,Q4
30,5,2171,L-180,Pre-Flight,"[79.20496894409938, 98.59627329192547, 110.509...",127.863014,1381,2759,1380
31,5,2171,L-60,Pre-Flight,"[224.4472049689441, 150.28571428571428, 119.00...",113.106425,1614,3235,671
32,5,2171,FD45,Mid-Flight,"[21.664596273291924, 32.91925465838509, 44.459...",135.290127,1413,2112,1995
33,5,2171,FD260,Mid-Flight,"[214.22360248447205, 91.63975155279503, 219.81...",134.600340,1230,2663,1627
34,5,2171,R+5,Post-Flight,"[75.1055900621118, 83.3913043478261, 95.801242...",87.765418,3009,2374,137
35,5,2171,R+105,Post-Flight,"[93.7888198757764, 114.3975155279503, 198.3478...",120.721879,1793,2600,1127
36,5,2171,R+180,Post-Flight,"[150.8944099378882, 141.87577639751552, 214.59...",112.335962,1558,3379,583


In [145]:
melted_quartiles_astro_df = pd.melt(
    quartiles_astro_df,
    id_vars=['astro number', 'astro id', 'timepoint', 'flight status', 'telo data', 'telo means'],
#     id_vars = [col for col in astro_df.columns if col != 'Q1' and col != 'Q2-3' and col != 'Q4'],
    var_name='relative Q',
    value_name='Q freq counts')

melted_quartiles_astro_df['Q freq counts'] = melted_quartiles_astro_df['Q freq counts'].astype('int64')
melted_quartiles_astro_df['astro id'] = melted_quartiles_astro_df['astro id'].astype('str')

In [146]:
melted_quartiles_astro_df[melted_quartiles_astro_df['astro id'] == '5163'].head(8)

,astro number,astro id,timepoint,flight status,telo data,telo means,relative Q,Q freq counts
0,1,5163,L-270,Pre-Flight,"[46.9762779819579, 78.38289341797528, 105.1119...",87.672120,Q1,1384
1,1,5163,L-180,Pre-Flight,"[102.38890745071835, 107.3170731707317, 61.944...",101.077756,Q1,848
2,1,5163,L-60,Pre-Flight,"[92.4323421316405, 144.537253591714, 135.59973...",128.599235,Q1,259
3,1,5163,FD90,Mid-Flight,"[89.22485800200468, 116.6388239224858, 116.221...",101.183129,Q1,472
4,1,5163,FD140,Mid-Flight,"[161.89442031406617, 152.82325425993986, 223.3...",129.851970,Q1,203
5,1,5163,R+7,Post-Flight,"[88.57333778817241, 86.86936184430337, 84.9315...",82.169298,Q1,1876
6,1,5163,R+60,Post-Flight,"[92.36551954560642, 75.97728032074842, 102.639...",100.116703,Q1,566
7,1,5163,R+180,Post-Flight,"[163.46475108586702, 157.11660541262947, 75.76...",115.403190,Q1,936


### Saving melted-quartile-count all astros dataframe for later retrieval

In [147]:
copy_melted_quartiles_astro_df = melted_quartiles_astro_df
copy_melted_quartiles_astro_df['telo data'] = copy_melted_quartiles_astro_df['telo data'].apply(lambda row: row.tolist())

melted_quartiles_astro_df.to_csv('../excel data/melted_quartiles_astro_df.csv', index = False)

### Reading in astronaut urine biochem data

In [67]:
urine_biochem_data = pd.read_csv('../excel data/urine_biochem_data.csv')
urine_biochem_data

,biochemistry,Pre,FD15,FD30,FD60,FD120,FD180,R+0 day 1,R+0 day 2,R+30 day 1,R+30 day 2,sample type
0,"8-OHdG, Urine ug/gCr",2.6 ± 1,3.1 ± 1,3 ± 1,3.6 ± 1.2,3 ± 1,2.5 ± 0.6,2.4 ± 0.8,2.4 ± 1,1.7 ± 0.7,1.8 ± 1,urine
1,"Copper, Urine umol/day",0.28 ± 0.17,0.24 ± 0.04,0.19 ± 0.02,0.24 ± 0.04,0.21 ± 0.05,0.24 ± 0.08,0.58 ± 0.71,0.25 ± 0.07,0.26 ± 0.1,0.34 ± 0.32,urine
2,PGF2-alpha ng/mg Cr,1.84 ± 1.2,2.22 ± 0.94,2.48 ± 1.05,2.92 ± 1.38,2.7 ± 1.26,3.12 ± 1.14,1.53 ± 1.06,1.59 ± 0.91,1.88 ± 1.12,1.72 ± 1.11,urine
3,"Selenium, Urine umol/day",1.02 ± 0.39,1.11 ± 0.35,1.08 ± 0.3,1.26 ± 0.54,1.14 ± 0.42,1.19 ± 0.42,0.88 ± 0.21,0.94 ± 0.27,0.96 ± 0.31,1.03 ± 0.41,urine


In [68]:
melt_urine_biochem_data = pd.melt(
    urine_biochem_data,
    id_vars=['biochemistry', 'sample type'],
    var_name='timepoint',
    value_name='measurement'
)

melt_urine_biochem_data.head(4)

,biochemistry,sample type,timepoint,measurement
0,"8-OHdG, Urine ug/gCr",urine,Pre,2.6 ± 1
1,"Copper, Urine umol/day",urine,Pre,0.28 ± 0.17
2,PGF2-alpha ng/mg Cr,urine,Pre,1.84 ± 1.2
3,"Selenium, Urine umol/day",urine,Pre,1.02 ± 0.39


In [69]:
def make_flight_status_via_timepoint(row):
    if 'Pre' in row:
        return 'Pre-Flight'
    elif 'FD' in row:
        return 'Mid-Flight'
    elif 'R' in row:
        return 'Post-Flight'

In [70]:
melt_urine_biochem_data['flight status'] = melt_urine_biochem_data['timepoint'].apply(lambda row: make_flight_status_via_timepoint(row))

In [71]:
def grab_number_remove_plusminus(row):
    if '±' in str(row):
        row = row.split('±')[0].strip()
        row = float(row)
        return row  

In [72]:
melt_urine_biochem_data['measured analyte'] = melt_urine_biochem_data['measurement'].apply(lambda row: grab_number_remove_plusminus(row))
melt_urine_biochem_data.head(4)

,biochemistry,sample type,timepoint,measurement,flight status,measured analyte
0,"8-OHdG, Urine ug/gCr",urine,Pre,2.6 ± 1,Pre-Flight,2.60
1,"Copper, Urine umol/day",urine,Pre,0.28 ± 0.17,Pre-Flight,0.28
2,PGF2-alpha ng/mg Cr,urine,Pre,1.84 ± 1.2,Pre-Flight,1.84
3,"Selenium, Urine umol/day",urine,Pre,1.02 ± 0.39,Pre-Flight,1.02


### Reading in astronaut blood biochem data

In [73]:
blood_biochem_data = pd.read_csv('../excel data/blood_biochem_data.csv')
blood_biochem_data.head(6)

,biochemistry,Pre,FD15,FD30,FD60,FD120,FD180,R+0,R+180,sample type
0,CCL2/MCP-1 pg/ml,83 ± 17,89 ± 21,94 ± 17,96 ± 26,96 ± 25,93 ± 22,182 ± 146,87 ± 23,blood
1,CCL3/MIP-1a pg/ml,438 ± 194,644 ± 214,652 ± 129,642 ± 218,489 ± 89,641 ± 247,310 ± 96,388 ± 105,blood
2,CCL4/MIP1B pg/ml,54 ± 17,75 ± 27,101 ± 34,75 ± 34,75 ± 37,70 ± 28,51 ± 16,48 ± 14,blood
3,CCL5/RANTES pg/ml,6902 ± 4393,15462 ± 2388,17986 ± 601,15022 ± 2726,14227 ± 2755,13636 ± 3964,3890 ± 3563,5569 ± 2539,blood
4,CXCL5/ENA-78 pg/ml,275 ± 549,1233 ± 967,1005 ± 426,1144 ± 823,1009 ± 503,919 ± 501,93 ± 74,134 ± 70,blood
5,CXCL8/IL-8 pg/ml,12 ± 5,23 ± 17,14 ± 7,18 ± 6,12 ± 2,19 ± 9,*,*,blood


In [74]:
blood_biochem_data.replace('*', np.NaN, inplace=True)

In [75]:
melt_blood_biochem_data = pd.melt(
    blood_biochem_data,
    id_vars=['biochemistry', 'sample type'],
    var_name='timepoint',
    value_name='measurement')

In [76]:
melt_blood_biochem_data['flight status'] = melt_blood_biochem_data['timepoint'].apply(lambda row: make_flight_status_via_timepoint(row))
melt_blood_biochem_data['measured analyte'] = melt_blood_biochem_data['measurement'].apply(lambda row: grab_number_remove_plusminus(row))
melt_blood_biochem_data.head(4)

,biochemistry,sample type,timepoint,measurement,flight status,measured analyte
0,CCL2/MCP-1 pg/ml,blood,Pre,83 ± 17,Pre-Flight,83.0
1,CCL3/MIP-1a pg/ml,blood,Pre,438 ± 194,Pre-Flight,438.0
2,CCL4/MIP1B pg/ml,blood,Pre,54 ± 17,Pre-Flight,54.0
3,CCL5/RANTES pg/ml,blood,Pre,6902 ± 4393,Pre-Flight,6902.0


In [77]:
melt_blood_biochem_data.rename(columns={'biochemistry':'biochemistry analyte'}, inplace=True)
melt_urine_biochem_data.rename(columns={'biochemistry':'biochemistry analyte'}, inplace=True)

In [78]:
melt_urine_biochem_data.to_csv('../excel data/melt_urine_biochem_data.csv', index=False)
melt_blood_biochem_data.to_csv('../excel data/melt_blood_biochem_data.csv', index=False)